In [3]:
import pandas as pd
import re

# Đọc dữ liệu từ file Excel
data = pd.read_excel('flight.xlsx')

# Hàm trích xuất mã sân bay từ tên sân bay
def extract_airport_code(airport_name):
    # Sử dụng biểu thức chính quy để lấy mã sân bay
    match = re.search(r'\((.*?)\)', airport_name)
    if match:
        return match.group(1)  # Trả về mã sân bay
    return None

# Áp dụng hàm vào cột airport_from để trích xuất mã sân bay
data['airport_code_from'] = data['airport_from'].apply(extract_airport_code)

print(data.head())

               id code         code_name f_code          f_time_from  \
0   vj16169846100   vj           Vietjet  VJ122  04:50:00 01/04/2021   
1  vna16169846200  vna  Vietnam Airlines  VN250  05:00:00 01/04/2021   
2   vj16169846231   vj           Vietjet  VJ176  05:25:00 01/04/2021   
3   qh16169846270   qh    Bamboo Airways  QH202  05:45:00 01/04/2021   
4   vj16169846302   vj           Vietjet  VJ120  06:00:00 01/04/2021   

             f_time_to  f_price    fees  total_price    from  \
0  07:10:00 01/04/2021   199000  650900       849900  TP HCM   
1  07:10:00 01/04/2021   299000  670000       969000  TP HCM   
2  07:35:00 01/04/2021   199000  650900       849900  TP HCM   
3  07:55:00 01/04/2021   399000  672000      1071000  TP HCM   
4  08:10:00 01/04/2021   199000  650900       849900  TP HCM   

         airport_from      to     airport_to            type airport_code_from  
0  Tân Sơn Nhất (SGN)  Hà Nội  Nội Bài (HAN)             Eco               SGN  
1  Tân Sơn Nhất (SGN

In [4]:
data['airport_code_to'] = data['airport_to'].apply(extract_airport_code)

In [5]:
print(data[['airport_to', 'airport_code_to']].head())

      airport_to airport_code_to
0  Nội Bài (HAN)             HAN
1  Nội Bài (HAN)             HAN
2  Nội Bài (HAN)             HAN
3  Nội Bài (HAN)             HAN
4  Nội Bài (HAN)             HAN


In [7]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Hàm tra cứu tọa độ sân bay từ mã sân bay
def get_airport_coordinates(airport_code):
    geolocator = Nominatim(user_agent="airport_distance_calculator")
    location = geolocator.geocode(f"{airport_code} airport")
    if location:
        return (location.latitude, location.longitude)
    return None

# Lấy tọa độ của sân bay SGN (Tân Sơn Nhất) và HAN (Nội Bài)
coords_sgn = get_airport_coordinates("SGN")
coords_han = get_airport_coordinates("HAN")

# Kiểm tra và tính khoảng cách nếu có tọa độ
if coords_sgn and coords_han:
    distance = geodesic(coords_sgn, coords_han).kilometers
    print(f"Khoảng cách giữa Tân Sơn Nhất (SGN) và Nội Bài (HAN) là {distance:.2f} km")
else:
    print("Không thể tra cứu tọa độ cho một hoặc cả hai sân bay.")

Khoảng cách giữa Tân Sơn Nhất (SGN) và Nội Bài (HAN) là 1154.51 km


In [8]:
data_cleaned = data.dropna(subset=['f_price', 'fees'])  # Xóa những dòng thiếu giá vé hoặc phí
data_cleaned['f_time_from'] = pd.to_datetime(data_cleaned['f_time_from'], format='%H:%M:%S %d/%m/%Y')
data_cleaned['f_time_to'] = pd.to_datetime(data_cleaned['f_time_to'], format='%H:%M:%S %d/%m/%Y')

In [9]:
vietjet_data = data_cleaned[data_cleaned['code_name'] == 'Vietjet']

print(vietjet_data.head())

              id code code_name f_code         f_time_from  \
0  vj16169846100   vj   Vietjet  VJ122 2021-04-01 04:50:00   
2  vj16169846231   vj   Vietjet  VJ176 2021-04-01 05:25:00   
4  vj16169846302   vj   Vietjet  VJ120 2021-04-01 06:00:00   
6  vj16169846383   vj   Vietjet  VJ132 2021-04-01 06:05:00   
8  vj16169846454   vj   Vietjet  VJ134 2021-04-01 07:00:00   

            f_time_to  f_price    fees  total_price    from  \
0 2021-04-01 07:10:00   199000  650900       849900  TP HCM   
2 2021-04-01 07:35:00   199000  650900       849900  TP HCM   
4 2021-04-01 08:10:00   199000  650900       849900  TP HCM   
6 2021-04-01 08:05:00   199000  650900       849900  TP HCM   
8 2021-04-01 09:10:00   409000  661900      1070900  TP HCM   

         airport_from      to     airport_to type airport_code_from  \
0  Tân Sơn Nhất (SGN)  Hà Nội  Nội Bài (HAN)  Eco               SGN   
2  Tân Sơn Nhất (SGN)  Hà Nội  Nội Bài (HAN)  Eco               SGN   
4  Tân Sơn Nhất (SGN)  Hà Nội  Nội 

In [11]:
# Mã hóa loại vé: Eco -> 0, SkyBoss -> 1
vietjet_data.loc[:, 'ticket_type'] = vietjet_data['type'].apply(lambda x: 0 if x == 'Eco' else 1)

print(vietjet_data[['type', 'ticket_type']].head())

  type  ticket_type
0  Eco            0
2  Eco            0
4  Eco            0
6  Eco            0
8  Eco            0
